In [8]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

from lib.data_preparation import DataPreparation
from lib.preprocessing_data import Preprocessing
from lib.vectorization import Vectorization


data_set = DataPreparation().load_data()
train_df = data_set.sample(frac=1, random_state=1).reset_index(drop=True, inplace=True)

split_index_1 = int(len(data_set) * 0.7)
split_index_2 = int(len(data_set) * 0.8)


train_df, test_df, validation_df = data_set[:split_index_1], data_set[split_index_1:split_index_2], data_set[split_index_2:]

len(train_df), len(test_df), len(validation_df)

(2490, 356, 712)

In [11]:
# df to X and y
def df_to_X_y(df):
    y = train_df['tweet'].to_numpy().astype(int)
    
